<a href="https://colab.research.google.com/github/yihimin/mbti_test/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,545 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123630 files and dire

In [ ]:
pip install pytrends

# 5개년도 노벨문학상 수상자를 이용한 언급량과 책 판매량의 상관관계

1. 데이터 준비

 1.1 노벨문학상 수상자 크롤링(위키피디아)


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# 1. URL 설정
url = "https://ko.wikipedia.org/wiki/노벨_문학상_수상자_목록"

# 2. HTTP 요청
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
response.raise_for_status()  # 요청 성공 확인

# 3. HTML 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 4. 테이블 가져오기 (13번째 테이블 선택)
tables = soup.find_all("table", {"class": "wikitable"})
if len(tables) < 13:
    print("테이블 개수가 충분하지 않습니다.")
    exit()

target_table = tables[12]  # 리스트 인덱스는 0부터 시작하므로 13번째는 인덱스 12

# 5. 데이터 추출 및 저장
rows = target_table.find_all("tr")  # 테이블 내 모든 행 가져오기

# 결과를 저장할 리스트
data = []

for row in rows[1:]:  # 헤더 제외
    columns = row.find_all("td")  # 각 행의 열 데이터
    if len(columns) >= 3:  # 열이 3개 이상인 경우만 처리
        year = columns[0].text.strip()  # 첫 번째 열: 연도
        winner = columns[2].text.strip()  # 세 번째 열: 수상자 이름

        # 2020년대 필터링
        if year.isdigit() and 2020 <= int(year) < 2030:
            data.append([year, winner])  # 리스트에 추가

# 6. CSV 파일 저장
output_file = "nobel_winners_2020s.csv"
with open(output_file, mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(["연도", "수상자"])  # 헤더 작성
    writer.writerows(data)  # 데이터 작성

print(f"데이터가 {output_file}에 저장되었습니다.")


데이터가 nobel_winners_2020s.csv에 저장되었습니다.



1.2 노벨문학상 수상자 언급량 비교(구글 트렌트)

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import time

# Google Trends API 연결
pytrends = TrendReq(hl='en-US', tz=360)

# 키워드 설정
keywords = ["Han Kang", "Jon Fosse", "Annie Ernaux", "Abdulrazak Gurnah", "Louise Glück"]

# 타임프레임 분할
timeframes = ["2020-01-01 2021-12-31", "2022-01-01 2024-12-31"]

# 데이터 저장용 DataFrame 초기화
all_data = pd.DataFrame()

# 데이터 요청
for keyword in keywords:
    for timeframe in timeframes:
        success = False
        attempts = 0
        while not success and attempts < 5:  # 최대 5회 재시도
            try:
                print(f"요청 중: {keyword}, 기간: {timeframe} (시도 {attempts + 1}/5)")
                pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo='', gprop='')
                data = pytrends.interest_over_time()
                if not data.empty:
                    print(f"{keyword} 데이터 수집 성공")
                    data = data[[keyword]]  # 현재 키워드 데이터만 선택
                    all_data = pd.concat([all_data, data], axis=1)  # 기존 데이터와 병합
                else:
                    print(f"{keyword}에 대한 데이터가 없습니다.")
                success = True
            except Exception as e:
                print(f"오류 발생: {e}")
                attempts += 1
                time.sleep(60)  # 실패 시 대기 시간 증가
        time.sleep(60)  # 각 키워드 요청 간 대기 시간

# 데이터 저장
output_file = "nobel_trends.csv"
all_data.to_csv(output_file, encoding="utf-8-sig")
print(f"데이터 저장 완료: {output_file}")



요청 중: Han Kang, 기간: 2020-01-01 2021-12-31 (시도 1/5)
Han Kang 데이터 수집 성공
요청 중: Han Kang, 기간: 2022-01-01 2024-12-31 (시도 1/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Han Kang, 기간: 2022-01-01 2024-12-31 (시도 2/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Han Kang, 기간: 2022-01-01 2024-12-31 (시도 3/5)


/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Han Kang 데이터 수집 성공
요청 중: Jon Fosse, 기간: 2020-01-01 2021-12-31 (시도 1/5)
Jon Fosse 데이터 수집 성공
요청 중: Jon Fosse, 기간: 2022-01-01 2024-12-31 (시도 1/5)


/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Jon Fosse 데이터 수집 성공
요청 중: Annie Ernaux, 기간: 2020-01-01 2021-12-31 (시도 1/5)
Annie Ernaux 데이터 수집 성공
요청 중: Annie Ernaux, 기간: 2022-01-01 2024-12-31 (시도 1/5)


...

Annie Ernaux 데이터 수집 성공
요청 중: Abdulrazak Gurnah, 기간: 2020-01-01 2021-12-31 (시도 1/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Abdulrazak Gurnah, 기간: 2020-01-01 2021-12-31 (시도 2/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Abdulrazak Gurnah, 기간: 2020-01-01 2021-12-31 (시도 3/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Abdulrazak Gurnah, 기간: 2020-01-01 2021-12-31 (시도 4/5)
Abdulrazak Gurnah 데이터 수집 성공
요청 중: Abdulrazak Gurnah, 기간: 2022-01-01 2024-12-31 (시도 1/5)


/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Abdulrazak Gurnah 데이터 수집 성공
요청 중: Louise Glück, 기간: 2020-01-01 2021-12-31 (시도 1/5)
Louise Glück 데이터 수집 성공
요청 중: Louise Glück, 기간: 2022-01-01 2024-12-31 (시도 1/5)
오류 발생: The request failed: Google returned a response with code 429
요청 중: Louise Glück, 기간: 2022-01-01 2024-12-31 (시도 2/5)


/usr/local/lib/python3.10/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Louise Glück 데이터 수집 성공
데이터 저장 완료: nobel_trends.csv


1.3 노벨문학상 수상자 책 판매순위

In [ ]:
import requests
from bs4 import BeautifulSoup

# 교보문고 검색 URL 예제
base_url = "https://search.kyobobook.co.kr/search?keyword={}&gbCode=TOT&target=total"
author = "한강"

response = requests.get(base_url.format(author), headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# 교보문고 데이터 크롤링 예제
books = soup.find_all("div", class_="title")
if books:
    for book in books[:5]:  # 5개의 데이터만 예제로 출력
        title = book.text.strip()
        print(f"책 제목: {title}")
else:
    print("책 데이터를 찾을 수 없습니다.")


책 데이터를 찾을 수 없습니다.


2. 데이터 정규화


   2.1 위 데이터를 수집하여 하나의 CSV 파일로 만듦



In [ ]:
winners_data.rename(columns={"수상자": "작가"}, inplace=True)


In [ ]:
# 열 이름 변경 확인
print(winners_data.columns)

Index(['연도', '작가'], dtype='object')


In [ ]:
print(sales_data.head())  # sales_data 데이터프레임 확인
print(sales_data.columns)  # 열 이름 확인


Empty DataFrame
Columns: [작가, 책 제목, 상세 링크, 판매 정보]
Index: []
Index(['작가', '책 제목', '상세 링크', '판매 정보'], dtype='object')


In [ ]:
# 임시 판매 데이터 생성
sales_data = pd.DataFrame({
    "작가": ["루이즈 글릭", "압둘라자크 구르나", "아니 에르노", "욘 포세", "한강"],
    "책 제목": ["책1", "책2", "책3", "책4", "책5"],
    "상세 링크": ["링크1", "링크2", "링크3", "링크4", "링크5"],
    "판매 정보": [500, 300, 400, 450, 600]
})


In [ ]:
# 병합 진행 (sales_data, winners_data)
merged_data = pd.merge(winners_data, sales_data, on="작가", how="inner")
print(merged_data.head())

KeyError: '작가'

In [ ]:
# 열 이름 정리 (열에서 .1 제거)
trends_data.columns = trends_data.columns.str.replace(".1", "", regex=False)

# 데이터프레임 변환 (long format)
trends_long = pd.melt(trends_data, id_vars=["date"], var_name="작가", value_name="언급량")

# 결측값 제거
trends_long.dropna(subset=["언급량"], inplace=True)

print(trends_long.head())


         date        작가   언급량
0  2019-12-29  Han Kang  17.0
1  2020-01-05  Han Kang  19.0
2  2020-01-12  Han Kang  19.0
3  2020-01-19  Han Kang  29.0
4  2020-01-26  Han Kang  35.0


In [ ]:
import pandas as pd

# CSV 파일 경로 설정
sales_file = "/content/nobel_books_sales.csv"
trends_file = "/content/nobel_trends.csv"
winners_file = "/content/nobel_winners_2020s.csv"

# 데이터 불러오기
sales_data = pd.read_csv(sales_file)
trends_data = pd.read_csv(trends_file)
winners_data = pd.read_csv(winners_file)

# 데이터 확인
print("Sales Data:")
print(sales_data.head())
print("\nTrends Data:")
print(trends_data.head())
print("\nWinners Data:")
print(winners_data.head())

# 데이터 병합 (예: 작가 이름을 기준으로 병합)
merged_data = pd.merge(winners_data, sales_data, on="작가", how="inner")
merged_data = pd.merge(merged_data, trends_data, on="작가", how="inner")

# 병합 결과 확인
print("\nMerged Data:")
print(merged_data.head())


Sales Data:
Empty DataFrame
Columns: [작가, 책 제목, 상세 링크, 판매 정보]
Index: []

Trends Data:
         date  Han Kang  Han Kang.1  Jon Fosse  Jon Fosse.1  Annie Ernaux  \
0  2019-12-29      17.0         NaN       25.0          NaN           9.0   
1  2020-01-05      19.0         NaN       28.0          NaN          11.0   
2  2020-01-12      19.0         NaN       23.0          NaN           9.0   
3  2020-01-19      29.0         NaN       31.0          NaN           8.0   
4  2020-01-26      35.0         NaN       22.0          NaN           8.0   

   Annie Ernaux.1  Abdulrazak Gurnah  Abdulrazak Gurnah.1  Louise Glück  \
0             NaN                0.0                  NaN           0.0   
1             NaN                0.0                  NaN           0.0   
2             NaN                0.0                  NaN           0.0   
3             NaN                0.0                  NaN           0.0   
4             NaN                0.0                  NaN           0.0   



KeyError: '작가'

2.2 데이터를 정규화 함

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 정규화 대상 열 선택
columns_to_normalize = ["언급량", "판매량"]  # 실제 열 이름으로 대체

# 정규화 도구 초기화
scaler = MinMaxScaler()

# 정규화 수행
merged_data[columns_to_normalize] = scaler.fit_transform(merged_data[columns_to_normalize])

print(merged_data.head())


In [ ]:
print(merged_data.describe())

In [ ]:
merged_data.to_csv("normalized_data.csv", index=False, encoding="utf-8-sig")
print("정규화된 데이터가 저장되었습니다.")

3. 데이터 분석

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# 1. 데이터 불러오기
# 언급량과 판매량 데이터를 병합한 CSV 파일 로드
data = pd.read_csv("normalized_data.csv")  # 정규화된 데이터를 저장한 파일 경로

# 데이터 확인
print(data.head())

# 2. Pearson 상관계수 계산
# 언급량과 판매량 열 이름을 확인 후 수정하세요.
mention_column = "언급량"  # 언급량 컬럼 이름
sales_column = "판매량"    # 판매량 컬럼 이름

# 상관계수 계산
correlation, p_value = pearsonr(data[mention_column], data[sales_column])
print(f"Pearson 상관계수: {correlation}")
print(f"P-value: {p_value}")

# 3. 시간적 추이를 선 그래프로 시각화
plt.figure(figsize=(10, 6))
plt.plot(data["날짜"], data[mention_column], label="언급량", marker="o")  # 날짜 기준으로 언급량
plt.plot(data["날짜"], data[sales_column], label="판매량", marker="s")    # 날짜 기준으로 판매량
plt.xlabel("날짜")
plt.ylabel("값")
plt.title("언급량과 판매량의 시간적 추이")
plt.legend()
plt.xticks(rotation=45)  # 날짜 라벨 회전
plt.tight_layout()
plt.show()

# 4. 언급량과 판매량 간의 상관관계를 산점도로 시각화
plt.figure(figsize=(8, 6))
plt.scatter(data[mention_column], data[sales_column], alpha=0.7)
plt.xlabel("언급량")
plt.ylabel("판매량")
plt.title("언급량과 판매량 간의 상관관계")
plt.grid()
plt.show()
